In [1]:
# Test code to implement ML attack on PUF via CRPsets

In [2]:
# Include Needed Libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import pandas as pd
%matplotlib inline

In [3]:
# Pull data
datafile = "CRPSets/CRPSets.xls"
df = pd.read_excel(datafile)

# Convert data to numpy array
numpy_array = df.to_numpy()

#DEBUG
#print(numpy_array.shape)

# Split into our training and testing sets
X = numpy_array[:, :-1] # Takes the first 64 columns as training data
t = numpy_array[:,-1] # takes the last column as test data

# Split our data so we have training and validation sets
# X_train is training data with t_train being the true values of this data
# X_test is validation data with t_test being the true values
X_train, X_test, t_train, t_test = train_test_split(X, t, 
                                                   test_size=0.2,
                                                   stratify=t,
                                                   random_state=1)
print(X_train.shape)
print(X_test.shape)


(9599, 64)
(2400, 64)


In [4]:
#**************************************
#********MODEL 1
#**************************************

# For Pipeline
# 1. SGD - Stochastic Gradient Descent, iterates until finding optimal solution
# 2. loss is our algorithm. Log_loss = Logistic Regression | perceptron = perceptron | hinge = SVM
# 3. Penalty is our regularization term, with Alpha being the amount of regularization

# Create and train a Logistic Regression pipeline based on the data (log_Loss is LR)
pipeLR = Pipeline([('scaler', StandardScaler()), # normalization (substracts the mean and divides by std)
                ('log_reg', SGDClassifier(loss='log_loss',penalty='elasticnet', learning_rate='optimal'))
 ])

pipeLR.fit(X_train, t_train)

# WE can use GridSearchCV as an Exhaustive search to find best parameters for each model
# Format is Start, step, end
params = {
    'log_reg__eta0': [0.001, 0.01, 1],
    'log_reg__alpha': [0.1, 1, 10, 100]
}

# GridSearchCV does Exhaustive search over specified parameters for a model
best_param_pipeLR = GridSearchCV(pipeLR, params, cv=10)


# Fit data using the model with best parameters
best_param_pipeLR.fit(X_train, t_train)

# Predict
y_train_LR = best_param_pipeLR.predict(X_train)
y_test_LR = best_param_pipeLR.predict(X_test)

# Commented out, but this is code to print performance
print('BEST PARAMETERS')
print(best_param_pipeLR.best_params_)
print('TRAINING SET PERFORMANCE:')
print(classification_report(t_train, y_train_LR),'\n\n')
print('TEST SET PERFORMANCE:')
print(classification_report(t_test, y_test_LR))
print('TRAINING SET CONFUSION MATRIX')
print(confusion_matrix(t_train, y_train_LR),'\n\n')
print('TEST SET CONFUSION MATRIX')
print(confusion_matrix(t_test, y_test_LR),'\n\n')
print('Accuracy Score')
print(accuracy_score(t_test, y_test_LR),'\n\n')



BEST PARAMETERS
{'log_reg__alpha': 0.1, 'log_reg__eta0': 0.001}
TRAINING SET PERFORMANCE:
              precision    recall  f1-score   support

           0       0.54      1.00      0.70      5195
           1       0.00      0.00      0.00      4404

    accuracy                           0.54      9599
   macro avg       0.27      0.50      0.35      9599
weighted avg       0.29      0.54      0.38      9599
 


TEST SET PERFORMANCE:
              precision    recall  f1-score   support

           0       0.54      1.00      0.70      1299
           1       0.00      0.00      0.00      1101

    accuracy                           0.54      2400
   macro avg       0.27      0.50      0.35      2400
weighted avg       0.29      0.54      0.38      2400

TRAINING SET CONFUSION MATRIX
[[5195    0]
 [4404    0]] 


TEST SET CONFUSION MATRIX
[[1299    0]
 [1101    0]] 


Accuracy Score
0.54125 




C:\Users\thoho\anaconda3\envs\Sandbox\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\thoho\anaconda3\envs\Sandbox\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\thoho\anaconda3\envs\Sandbox\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\thoho\a

In [5]:
#**************************************
# MODEL 2: Perceptron
#**************************************
# Create a Pipeline to scale data, conduct MLP. Fit for our data
pipeMLP = Pipeline([('scaler', StandardScaler()), # normalization (substracts the mean and divides by std)
                ('log_reg', SGDClassifier(loss='perceptron',penalty='elasticnet',learning_rate='optimal'))])

pipeMLP.fit(X_train, t_train)


# WE can use GridSearchCV as an Exhaustive search to find best parameters for each model
# Format is Start, step, end
params = {
    'log_reg__eta0': [0.001, 0.01, 0.1],
    'log_reg__alpha': [0.1, 1, 10, 100]
}

# GridSearchCV does Exhaustive search over specified parameters for a model
best_param_pipeMLP = GridSearchCV(pipeMLP, params, cv=10)


# Fit data using the model with best parameters
best_param_pipeMLP.fit(X_train, t_train)

# Predict
y_train_MLP = best_param_pipeMLP.predict(X_train)
y_test_MLP = best_param_pipeMLP.predict(X_test)

# Commented out, but this is code to print performance
print('BEST PARAMETERS')
print(best_param_pipeMLP.best_params_)
print('TRAINING SET PERFORMANCE:')
print(classification_report(t_train, y_train_MLP),'\n\n')
print('TEST SET PERFORMANCE:')
print(classification_report(t_test, y_test_MLP))
print('TRAINING SET CONFUSION MATRIX')
print(confusion_matrix(t_train, y_train_MLP),'\n\n')
print('TEST SET CONFUSION MATRIX')
print(confusion_matrix(t_test, y_test_MLP),'\n\n')
print('Accuracy Score')
print(accuracy_score(t_test, y_test_MLP),'\n\n')

BEST PARAMETERS
{'log_reg__alpha': 0.1, 'log_reg__eta0': 0.1}
TRAINING SET PERFORMANCE:
              precision    recall  f1-score   support

           0       0.54      1.00      0.70      5195
           1       0.00      0.00      0.00      4404

    accuracy                           0.54      9599
   macro avg       0.27      0.50      0.35      9599
weighted avg       0.29      0.54      0.38      9599
 


TEST SET PERFORMANCE:
              precision    recall  f1-score   support

           0       0.54      1.00      0.70      1299
           1       0.00      0.00      0.00      1101

    accuracy                           0.54      2400
   macro avg       0.27      0.50      0.35      2400
weighted avg       0.29      0.54      0.38      2400

TRAINING SET CONFUSION MATRIX
[[5195    0]
 [4404    0]] 


TEST SET CONFUSION MATRIX
[[1299    0]
 [1101    0]] 


Accuracy Score
0.54125 




C:\Users\thoho\anaconda3\envs\Sandbox\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\thoho\anaconda3\envs\Sandbox\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\thoho\anaconda3\envs\Sandbox\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\thoho\a

In [6]:
#**************************************
# Model 3: SVM
#**************************************
# Create a Pipeline to scale data, conduct Support Vector Machine. Fit for our data
pipeSVC = Pipeline([('scaler', StandardScaler()), # normalization (substracts the mean and divides by std)
                ('log_reg', SGDClassifier(loss='hinge',penalty='elasticnet',learning_rate='optimal'))])

pipeSVC.fit(X_train, t_train)


# WE can use GridSearchCV as an Exhaustive search to find best parameters for each model
# Format is Start, step, end
params = {
    'log_reg__eta0': [0.001, 0.01, 0.1],
    'log_reg__alpha': [0.1, 1, 10, 100]
}

# GridSearchCV does Exhaustive search over specified parameters for a model
best_param_pipeSVC = GridSearchCV(pipeSVC, params, cv=10)


# Fit data using the model with best parameters
best_param_pipeSVC.fit(X_train, t_train)

# Predict
y_train_SVC = best_param_pipeSVC.predict(X_train)
y_test_SVC = best_param_pipeSVC.predict(X_test)

# Commented out, but this is code to print performance
print('BEST PARAMETERS')
print(best_param_pipeSVC.best_params_)
print('TRAINING SET PERFORMANCE:')
print(classification_report(t_train, y_train_SVC),'\n\n')
print('TEST SET PERFORMANCE:')
print(classification_report(t_test, y_test_SVC))
print('TRAINING SET CONFUSION MATRIX')
print(confusion_matrix(t_train, y_train_SVC),'\n\n')
print('TEST SET CONFUSION MATRIX')
print(confusion_matrix(t_test, y_test_SVC),'\n\n')
print('Accuracy Score')
print(accuracy_score(t_test, y_test_SVC),'\n\n')

BEST PARAMETERS
{'log_reg__alpha': 0.1, 'log_reg__eta0': 0.001}
TRAINING SET PERFORMANCE:
              precision    recall  f1-score   support

           0       0.54      1.00      0.70      5195
           1       0.00      0.00      0.00      4404

    accuracy                           0.54      9599
   macro avg       0.27      0.50      0.35      9599
weighted avg       0.29      0.54      0.38      9599
 


TEST SET PERFORMANCE:
              precision    recall  f1-score   support

           0       0.54      1.00      0.70      1299
           1       0.00      0.00      0.00      1101

    accuracy                           0.54      2400
   macro avg       0.27      0.50      0.35      2400
weighted avg       0.29      0.54      0.38      2400

TRAINING SET CONFUSION MATRIX
[[5195    0]
 [4404    0]] 


TEST SET CONFUSION MATRIX
[[1299    0]
 [1101    0]] 


Accuracy Score
0.54125 




C:\Users\thoho\anaconda3\envs\Sandbox\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\thoho\anaconda3\envs\Sandbox\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\thoho\anaconda3\envs\Sandbox\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\thoho\a